In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
from imageio import imread
from skimage.transform import resize
from scipy.spatial import distance
from keras.models import load_model
import requests
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
image_dir_basepath = 'train/'
names = [str(i).zfill(3) for i in range (0,100)]
image_size = 160


In [3]:
print((names))

['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099']


In [4]:
model_path = '../model/keras/model/facenet_keras.h5'
model = load_model(model_path)

/usr/local/lib/python2.7/dist-packages/keras/models.py:251: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [6]:
def load_and_align_images(filepaths, margin):
    cascade = cv2.CascadeClassifier(cascade_path)
    
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)

        faces = cascade.detectMultiScale(img,
                                         scaleFactor=1.1,
                                         minNeighbors=3)
        #print(filepath)
        #print(faces)
        (x, y, w, h) = faces[0]
        cropped = img[y-margin//2:y+h+margin//2,
                      x-margin//2:x+w+margin//2, :]
        aligned = resize(cropped, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
            
    return np.array(aligned_images)

In [7]:
def load_and_align_images(filepaths, margin):
    aligned_images = []
    for filepath in filepaths:
        img = cv2.imread(filepath)
        url='http://192.168.2.24:9010'
        files = {'im': open(filepath, 'rb')}
        res=requests.post(url,files=files)
        #print res.text
        a=res.text
        a=a.split('\n')
        print(len(a))
        boxes=[]
        for x in a:
            #print(x)
            x=x.strip()
            x=x.strip('[')
            x=x.strip(']')
            print(x)
            boxes.append(x)
        print(((boxes[0])))
        dic={}
    

        for i,box in enumerate(boxes):
            box=box.split(',')
            print(box)
            numbers=[]
            if 'e' in box[4]:
                    p=box[4].split('e')
                    print(p)
                    num1=float(p[0])/(10**int(p[1][1:]))
            else:
                    num1=(float(box[4]))
            if num1<0.9:
                continue
                
            for t in box[0:4]:
           
                if 'e' in t:
                    p=t.split('e+')
                    print(p)
                    num=float(p[0])*(10**int(p[1]))
                    numbers.append(num)
                else:
                    numbers.append(float(t))
            dic[i]=numbers
        for k,box in dic.items():
            box1=[box[0],box[1],box[2]-box[0],box[3]-box[1]]
            if box1[2]>=box1[3]:
                diff=box1[2]-box1[3]
                box1[3]=box1[2]
                box1[1]=box1[1]-diff/2
            else:
                diff=box1[3]-box1[2]
                box1[2]=box1[3]
                box1[0]=box1[0]-diff/2
            print(box1)

            (x, y, w, h) = box1
            margin=0
            
            cropped = img[int(y)-margin//2:int(y)+int(h)+margin//2,int(x)-margin//2:int(x)+int(w)+margin//2,:]

            aligned = resize(cropped, (image_size, image_size), mode='reflect')
            aligned_images.append(aligned)
    print(len(aligned_images))  
    return np.array(aligned_images)

In [8]:
def load_and_align_images(filepaths, margin):
    aligned_images = []
    for filepath in filepaths:
        img = cv2.imread(filepath)
        url='http://127.0.0.1:9004'
        files = {'im': open(filepath, 'rb')}
        res=requests.post(url,files=files)
        #print res.text
        boxes=eval(res.text)
        
        
        for box in boxes:
            if box[4]<0.9:
                continue
            box1=[box[0],box[1],box[2]-box[0],box[3]-box[1]]
            if box1[2]>=box1[3]:
                diff=box1[2]-box1[3]
                box1[3]=box1[2]
                box1[1]=box1[1]-diff/2
            else:
                diff=box1[3]-box1[2]
                box1[2]=box1[3]
                box1[0]=box1[0]-diff/2
            print(box1)

            (x, y, w, h) = box1
            margin=0
            
            cropped = img[int(y)-margin//2:int(y)+int(h)+margin//2,int(x)-margin//2:int(x)+int(w)+margin//2,:]

            aligned = resize(cropped, (image_size, image_size), mode='reflect')
            aligned_images.append(aligned)
    print(len(aligned_images))  
    return np.array(aligned_images)

In [9]:
def calc_embs(filepaths, margin=10, batch_size=1):
    aligned_images = prewhiten(load_and_align_images(filepaths, margin))
    pd = []
    for start in range(0, len(aligned_images), batch_size):
        pd.append(model.predict_on_batch(aligned_images[start:start+batch_size]))
    embs = l2_normalize(np.concatenate(pd))

    return embs

In [10]:
def calc_dist(img_name0, img_name1):
    return distance.euclidean(data[img_name0]['emb'], data[img_name1]['emb'])

def calc_dist_plot(img_name0, img_name1):
    print(calc_dist(img_name0, img_name1))
    plt.subplot(1, 2, 1)
    plt.imshow(imread(data[img_name0]['image_filepath']))
    plt.subplot(1, 2, 2)
    plt.imshow(imread(data[img_name1]['image_filepath']))

In [11]:

data = {}
for name in names:
    image_dirpath = image_dir_basepath + name
    
    image_filepaths = [os.path.join(image_dirpath, f) for f in os.listdir(image_dirpath)]
    print(image_filepaths)
    embs = calc_embs(image_filepaths)
    for i in range(len(image_filepaths)):
        data['{}{}'.format(name, i)] = {'image_filepath' : image_filepaths[i],
                                        'emb' : embs[i]}

['train/000/000_3.bmp', 'train/000/000_1.bmp', 'train/000/000_0.bmp', 'train/000/000_4.bmp']
[302.79571533203125, 178.64138793945312, 107.12905883789062, 107.12905883789062]
[309.5580062866211, 174.9702606201172, 106.62037658691406, 106.62037658691406]
[226.56659698486328, 133.91722106933594, 157.88575744628906, 157.88575744628906]
[256.2802200317383, 150.2062530517578, 150.0962371826172, 150.0962371826172]
4
['train/001/001_0.bmp', 'train/001/001_3.bmp', 'train/001/001_1.bmp', 'train/001/001_4.bmp']
[260.29579162597656, 140.26205444335938, 138.66070556640625, 138.66070556640625]
[246.64953231811523, 123.0672836303711, 152.50830841064453, 152.50830841064453]
[263.1142883300781, 146.373046875, 127.69329833984375, 127.69329833984375]
[262.5580749511719, 149.51260375976562, 125.71377563476562, 125.71377563476562]
4
['train/002/002_4.bmp', 'train/002/002_3.bmp', 'train/002/002_1.bmp', 'train/002/002_0.bmp']
[300.96981048583984, 161.8440704345703, 103.54319763183594, 103.54319763183594]
[24

['train/020/020_4.bmp', 'train/020/020_0.bmp', 'train/020/020_1.bmp', 'train/020/020_3.bmp']
[294.28736877441406, 148.2401123046875, 136.46124267578125, 136.46124267578125]
[279.2188415527344, 145.264404296875, 129.0634765625, 129.0634765625]
[280.66178131103516, 159.88461303710938, 94.47911071777344, 94.47911071777344]
[280.4318542480469, 165.33889770507812, 93.53445434570312, 93.53445434570312]
4
['train/021/021_3.bmp', 'train/021/021_0.bmp', 'train/021/021_4.bmp', 'train/021/021_1.bmp']
[345.78324127197266, 142.8606414794922, 118.13349914550781, 118.13349914550781]
[327.9721221923828, 142.65164184570312, 118.53604125976562, 118.53604125976562]
[326.1005096435547, 140.65402221679688, 117.36553955078125, 117.36553955078125]
[307.3875427246094, 170.60867309570312, 88.5704345703125, 88.5704345703125]
4
['train/022/022_3.bmp', 'train/022/022_4.bmp', 'train/022/022_1.bmp', 'train/022/022_0.bmp']
[277.37843322753906, 176.31015014648438, 92.48849487304688, 92.48849487304688]
[253.9786911010

['train/040/040_3.bmp', 'train/040/040_1.bmp', 'train/040/040_0.bmp', 'train/040/040_4.bmp']
[279.87215423583984, 130.3179473876953, 143.0430145263672, 143.0430145263672]
[309.47396087646484, 136.03282165527344, 116.51210021972656, 116.51210021972656]
[297.57896423339844, 133.28823852539062, 132.97991943359375, 132.97991943359375]
[296.14952087402344, 133.83151245117188, 132.56329345703125, 132.56329345703125]
4
['train/041/041_1.bmp', 'train/041/041_3.bmp', 'train/041/041_0.bmp', 'train/041/041_4.bmp']
[301.9001235961914, 165.5536346435547, 107.85990905761719, 107.85990905761719]
[262.3493423461914, 156.7457733154297, 135.97251892089844, 135.97251892089844]
[262.4063720703125, 157.1297607421875, 137.69183349609375, 137.69183349609375]
[300.22110748291016, 162.66017150878906, 110.14689636230469, 110.14689636230469]
4
['train/042/042_3.bmp', 'train/042/042_0.bmp', 'train/042/042_1.bmp', 'train/042/042_4.bmp']
[282.62623596191406, 175.49932861328125, 87.17803955078125, 87.17803955078125]

['train/060/060_1.bmp', 'train/060/060_3.bmp', 'train/060/060_0.bmp', 'train/060/060_4.bmp']
[296.4272918701172, 184.15158081054688, 85.57476806640625, 85.57476806640625]
[293.79620361328125, 184.11776733398438, 86.38180541992188, 86.38180541992188]
[286.8329162597656, 178.63613891601562, 103.77310180664062, 103.77310180664062]
[285.83917236328125, 184.00985717773438, 101.98007202148438, 101.98007202148438]
4
['train/061/061_1.bmp', 'train/061/061_4.bmp', 'train/061/061_0.bmp', 'train/061/061_3.bmp']
[293.28511810302734, 180.0695037841797, 144.8967742919922, 144.8967742919922]
[285.4657287597656, 169.18975830078125, 157.12973022460938, 157.12973022460938]
[267.13231658935547, 148.57850646972656, 209.4613494873047, 209.4613494873047]
[270.70047760009766, 154.9288787841797, 193.33351135253906, 193.33351135253906]
4
['train/062/062_0.bmp', 'train/062/062_4.bmp', 'train/062/062_3.bmp', 'train/062/062_1.bmp']
[246.43148040771484, 151.0891876220703, 156.5737762451172, 156.5737762451172]
[236

[277.95372772216797, 102.4403076171875, 149.3370819091797, 149.3370819091797]
[299.9759826660156, 107.338623046875, 139.84164428710938, 139.84164428710938]
[298.03644943237305, 126.0327377319336, 116.8926773071289, 116.8926773071289]
4
['train/081/081_4.bmp', 'train/081/081_3.bmp', 'train/081/081_1.bmp', 'train/081/081_0.bmp']
[316.4781188964844, 129.34512329101562, 115.56982421875, 115.56982421875]
[321.52552032470703, 128.10110473632812, 122.61845397949219, 122.61845397949219]
[313.9697036743164, 139.37991333007812, 106.16664123535156, 106.16664123535156]
[315.969482421875, 128.81536865234375, 116.95138549804688, 116.95138549804688]
4
['train/082/082_4.bmp', 'train/082/082_1.bmp', 'train/082/082_3.bmp', 'train/082/082_0.bmp']
[298.7204360961914, 106.8017578125, 107.21488952636719, 107.21488952636719]
[306.4502944946289, 114.83226013183594, 95.96122741699219, 95.96122741699219]
[307.9097709655762, 99.85311126708984, 110.48102569580078, 110.48102569580078]
[299.5430679321289, 107.07148

In [12]:
def faceRecognition(image_path,data):
    """
    Implements face recognition for the happy house by finding who is the person on the image_path image.
    
    Arguments:
    image_path -- path to an image
    database -- database containing image encodings along with the name of the person on the image
    model -- your Inception model instance in Keras
    
    Returns:
    min_dist -- the minimum distance between image_path encoding and the encodings from the database
    identity -- string, the name prediction for the person on image_path
    """
    embs = calc_embs(image_path)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    
    for name in names:
        dist_array=[]
        for k,v in data.items():
            if k.startswith(name):
                dist_array.append(calc_dist_self(embs,k))
        a=0       
        for i in dist_array:
            a+=i
        dist=a/len(dist_array)
            
        if dist<min_dist:
            min_dist = dist
            identity = name

    ### END CODE HERE ###
    print(min_dist)
    if min_dist > 0.9:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
                                                                                                                                                                               
    return min_dist, identity

def calc_dist_self(emb, img_name1):
    return distance.euclidean(emb, data[img_name1]['emb'])


In [13]:
print(data)

{'0831': {'image_filepath': 'train/083/083_0.bmp', 'emb': array([ 0.04094186, -0.13777964, -0.17209409, -0.05267674,  0.02215357,
       -0.14446442, -0.06456761,  0.0610209 ,  0.0875897 , -0.1360912 ,
        0.07985539,  0.05376247,  0.01630488, -0.23908065,  0.02916278,
       -0.02134449,  0.10348579,  0.08685326, -0.02817625,  0.02666192,
       -0.03478233,  0.04826825, -0.15361433, -0.01831054,  0.04176205,
        0.09608343,  0.04822895,  0.0539532 , -0.01394513, -0.11619014,
        0.00086014,  0.22363503,  0.07022306, -0.0538785 , -0.02548464,
        0.08686219, -0.15001118, -0.01824296, -0.10630322, -0.02981273,
        0.01718326,  0.12862912,  0.04175178, -0.10367235, -0.06077052,
       -0.01685658,  0.10567505,  0.02793384, -0.10291052,  0.08197409,
        0.12007336, -0.03669238,  0.04255519,  0.01277309,  0.12830405,
       -0.04749859, -0.15002665, -0.00362987, -0.07775255, -0.14131676,
        0.00787816, -0.12261087, -0.07650393,  0.15992466,  0.26437113,
      

In [19]:
image_dir_basepath = 'test/'
count=0
for name in names:
    image_dirpath = image_dir_basepath + name
    for f in os.listdir(image_dirpath):
        image_filepath = os.path.join(image_dirpath, f)
        #print(image_filepath)
        dist,predict=faceRecognition([str(image_filepath)],data)
        print (predict)
        print (name)
        if predict==name and dist<0.7:
            count+=1
print(float(count)/len(names))

[230.451904296875, 159.03355407714844, 154.6696319580078, 154.6696319580078]
1
0.712534934282
it's 000, the distance is 0.712534934282
000
000
[298.2335739135742, 154.9866485595703, 113.07807922363281, 113.07807922363281]
1
0.427921261638
it's 001, the distance is 0.427921261638
001
001
[237.2492904663086, 154.4868927001953, 138.4458465576172, 138.4458465576172]
1
0.589203998446
it's 002, the distance is 0.589203998446
002
002
[220.3667221069336, 126.04582214355469, 188.6007843017578, 188.6007843017578]
1
0.550939664245
it's 003, the distance is 0.550939664245
003
003
[261.66981506347656, 166.40444946289062, 118.26882934570312, 118.26882934570312]
1
0.48917298764
it's 004, the distance is 0.48917298764
004
004
[194.3960723876953, 118.11259460449219, 176.66123962402344, 176.66123962402344]
1
0.560652785003
it's 005, the distance is 0.560652785003
005
005
[282.5542221069336, 193.5863800048828, 115.87309265136719, 115.87309265136719]
1
0.545279584825
it's 006, the distance is 0.5452795848

[256.2388229370117, 151.4017791748047, 126.29072570800781, 126.29072570800781]
1
0.521903239191
it's 057, the distance is 0.521903239191
057
057
[277.8074188232422, 219.94967651367188, 94.21102905273438, 94.21102905273438]
1
0.416739910841
it's 058, the distance is 0.416739910841
058
058
[254.9911880493164, 183.2329559326172, 120.17884826660156, 120.17884826660156]
1
0.500821709633
it's 059, the distance is 0.500821709633
059
059
[278.88221740722656, 176.57339477539062, 110.32058715820312, 110.32058715820312]
1
0.605782717466
it's 060, the distance is 0.605782717466
060
060
[298.93521881103516, 176.3262481689453, 158.1225128173828, 158.1225128173828]
1
0.569838434458
it's 061, the distance is 0.569838434458
061
061
[256.3285675048828, 141.97793579101562, 154.400390625, 154.400390625]
1
0.62003429234
it's 062, the distance is 0.62003429234
062
062
[283.4459457397461, 149.4446563720703, 137.9445343017578, 137.9445343017578]
1
0.641658350825
it's 063, the distance is 0.641658350825
063
06